In [1]:
################ Imports ##########################
import pandas as pd
import macrobond_data_api as mda
from datetime import datetime
from dateutil.tz import tzutc
# from macrobond_data_api.common.enums import SeriesFrequency
import plotly.express as px
import matplotlib.pyplot as plt
import numpy as np
# pip install dash 
from dash import Dash, html, dcc 

In [2]:
##################### Start Data Retrieval & Processing Code ################
# Declare your time series of interest, how you want them labeled in the release table and the units 
tsoi_list = ['ussurv1044', 'uslama1060', 'uslama1849', 'uscons1146', 'usfcst2436', 'usrate1284', 'usnaac0381', 'uspric0006', 'usnaac3021', 'usnaac3022', 'ussurv1055', 'usrate0001', 'ustrad0068', 'usprod1385']
labels_list = ['ISM Services PMI', 'Nonfarm Payrolls', 'Unemployment Rate', 'Building Permits', 'FOMC Economic Projections', 'Fed Press Conference', 'GDP Q/Q Growth Rate', 'Core PCE M/M', 'Personal Income Monthly', 'Personal Spending Monthly', 'ISM Manufacturing PMI', 'Fed Interest Rate Decision', 'Retail Sales', 'Durable Goods']
unit_list = ['percent', 'million', 'percent', 'million', 'percent', 'percent', 'percent', 'index', 'trillion', 'trillion', 'index', 'percent', 'billion', 'billion']
universe = mda.get_series(tsoi_list)

# Creating data/metadata universe
metadata_universe = pd.DataFrame({
    'PrimName': [x.metadata['PrimName'] for x in universe],
    'LastModifiedTimeStamp': [x.metadata['LastModifiedTimeStamp'] for x in universe],
    'Release': [x.metadata['Release'] if 'Release' in x.metadata else np.nan for x in universe],
    'FullDescription': [x.metadata['FullDescription'] for x in universe],
    'Previous Value':[y.to_dict()['Values'][-1] for y in universe]
})
if metadata_universe.Release.isna().any():
    print("Warning: The following series do not have a release calendar, and are removed from your watchlist.")
    # display(metadata_universe[metadata_universe.Release.isna()])
    metadata_universe = metadata_universe[metadata_universe.Release.notna()]
metadata_universe['Title'] = labels_list
metadata_universe['Unit'] = unit_list

# Getting release metadata for the release associated with each time series of interest
release_info = mda.get_entities(metadata_universe['Release'].drop_duplicates())
release = pd.DataFrame({
    'NextReleaseEventReferencePeriod': [x.metadata['NextReleaseEventReferencePeriod'] if 'NextReleaseEventReferencePeriod' in x.metadata else np.nan for x in release_info],
    'Release_PrimName': [x.metadata['PrimName'] for x in release_info],
    'Release_FullDescription': [x.metadata['FullDescription'] for x in release_info],
    'LastReleaseEventTime': [x.metadata['LastReleaseEventTime'] if 'LastReleaseEventTime' in x.metadata else np.nan for x in release_info],
    'NextReleaseEventTime': [x.metadata['NextReleaseEventTime'] if 'NextReleaseEventTime' in x.metadata else np.nan for x in release_info]
})
if release.NextReleaseEventTime.isnull().any():
    print("The following series do not have a next release event time")
    print(release[release.NextReleaseEventTime.isnull()])
    release = release[release.NextReleaseEventTime.notna()]

In [ ]:
################### Data Cleaning and Dataframe Manipulation Code ########################
full_df = pd.merge(metadata_universe, release, left_on='Release', right_on='Release_PrimName')

# Creating cleaner dataframe for further processing
full_df = full_df[['NextReleaseEventTime', 'FullDescription', 'Previous Value', 'Title', 'Unit']]
full_df['time'] = full_df['NextReleaseEventTime'].apply(lambda x: None if pd.isnull(x) else x.strftime("%I:%M %p"))
full_df['date'] = full_df['NextReleaseEventTime'].apply(lambda x: None if pd.isnull(x) else x.strftime("%D"))
full_df['Region'] = full_df['FullDescription'].apply(lambda x: x.split(',')[0])
# For formatting in the graphic
full_df['space_1'] = '        '
full_df['space_2'] = '        '
full_df['space_3'] = '        '

# Adjusting value to include units
for i in range(len(full_df)):
    old_value = full_df['Previous Value'].loc[i]
    if full_df['Unit'].loc[i] == 'percent':
        new_value = str(round(old_value, 2)) + '%'
        full_df['Previous Value'].loc[i] = new_value
    elif full_df['Unit'].loc[i] == 'million':
        new_value = str(round(old_value/1000000, 1)) + 'M'
        full_df['Previous Value'].loc[i] = new_value
    elif full_df['Unit'].loc[i] == 'trillion':
        new_value = str(round(old_value/1000000000000, 1)) + 'T'
        full_df['Previous Value'].loc[i] = new_value
    elif full_df['Unit'].loc[i] == 'billion':
        new_value = str(round(old_value/1000000000, 1)) + 'B'
        full_df['Previous Value'].loc[i] = new_value
    elif full_df['Unit'].loc[i]=='index':
        new_value = str(round(old_value, 1))
        full_df['Previous Value'].loc[i] = new_value

#Sorting releases by release time and creating final dataframe for visualization:
full_df = full_df.sort_values(by='NextReleaseEventTime')
final_df = full_df[['time', 'space_1', 'Region', 'space_2', 'Title', 'space_3', 'Previous Value']]

# Creating data dictionary to group the data (indicator release information and previous value) by date: 
release_data_dict = {}
i=1
for date in list(full_df['date'].unique()):
    if date is None:
        release_data_dict['sub_df_'+str(i)] = final_df.loc[full_df['date'].isna()]
    else:    
        release_data_dict['sub_df_'+str(i)] = final_df.loc[full_df['date']==date]
    i+=1

# Adjusting column names for formatting in the graphic:
for key in release_data_dict.keys():
    release_data_dict[key] =release_data_dict[key].rename({'time':'________', 'space_1':'______', 'Region':'___________', 'space_2':'_______', 'Title':'_____________________________', 'space_3':'_________'}, axis=1)

# Creating title dictionary to store title for each table (aka the date of the release that is displayed)
title_dict = {}
i=1
for date in list(full_df['date'].unique()):
    df = full_df.loc[full_df['date']==date]
    df=df.reset_index().drop('index', axis=1)
    if date is None: 
        title_dict['title_'+str(i)] = 'No Release Date Mapped'
    else:
        title_dict['title_'+str(i)] = str(df['NextReleaseEventTime'].loc[len(df)-1].strftime("%A")) + ' ' + str(df['NextReleaseEventTime'].loc[len(df)-1].strftime("%b")) + ' ' + str(df['NextReleaseEventTime'].loc[len(df)-1].day) + ', ' + str(df['NextReleaseEventTime'].loc[len(df)-1].year)
    i+=1

# To return the number of different tables user needs to add to the graphic (in app.layout) below:
len(full_df['date'].unique())

########################## Start Visualization Code ######################
def generate_table(data_frame, max_rows=10):
    return html.Table([
        html.Thead(
            html.Tr([html.Th(col) for col in data_frame.columns])
        ),
        html.Tbody([
            html.Tr([
                html.Td(data_frame.iloc[i][col]) for col in data_frame.columns
            ]) for i in range(min(len(data_frame), max_rows))
        ])
    ])

# Creating the graphic and adding 9 tables to the layout because there were 9 different release dates for the 14 tickers/time series inputed
app = Dash(__name__)
app.layout = html.Div([
    html.H3(children=title_dict['title_1']),
    generate_table(release_data_dict['sub_df_1']),
    html.H3(children=title_dict['title_2']),
    generate_table(release_data_dict['sub_df_2']),
    html.H3(children=title_dict['title_3']),
    generate_table(release_data_dict['sub_df_3']),
    html.H3(children=title_dict['title_4']),
    generate_table(release_data_dict['sub_df_4']),
    html.H3(children=title_dict['title_5']),
    generate_table(release_data_dict['sub_df_5']),
    html.H3(children=title_dict['title_6']),
    generate_table(release_data_dict['sub_df_6']),
    html.H3(children=title_dict['title_7']),
    generate_table(release_data_dict['sub_df_7']),
    html.H3(children=title_dict['title_8']),
    generate_table(release_data_dict['sub_df_8']),
    html.H3(children=title_dict['title_9']),
    generate_table(release_data_dict['sub_df_9']),
])

if __name__ == '__main__':
    app.run(debug=False)

# In order to open the figure (it will open in a local browser) ctrl+click on the http: link in the terminal after 'Dash is running on '
# Press crtl+c in the terminal to break the connection
    
